<a href="https://colab.research.google.com/github/dinuwanfernando/10822750-final-year-project/blob/main/loan_prediction_version2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Loan Prediction Model - Version 2**

In [1]:
#Importing libraries
import pandas as pd
import numpy as np
import math
import datetime
from datetime import datetime
import seaborn as sns
import xgboost as xgb
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.varmax import VARMAX
from statsmodels.tsa.api import VAR
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV
from math import sqrt
import matplotlib.pyplot as plt
!pip install prophet
from prophet import Prophet

In [2]:
df= pd.read_csv('/content/train.csv')
print(df)

      Loan_ID  Gender Married Dependents     Education Self_Employed  \
0    LP001002    Male      No          0      Graduate            No   
1    LP001003    Male     Yes          1      Graduate            No   
2    LP001005    Male     Yes          0      Graduate           Yes   
3    LP001006    Male     Yes          0  Not Graduate            No   
4    LP001008    Male      No          0      Graduate            No   
..        ...     ...     ...        ...           ...           ...   
609  LP002978  Female      No          0      Graduate            No   
610  LP002979    Male     Yes         3+      Graduate            No   
611  LP002983    Male     Yes          1      Graduate            No   
612  LP002984    Male     Yes          2      Graduate            No   
613  LP002990  Female      No          0      Graduate           Yes   

     ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
0               5849                0.0         NaN           

In [3]:
df_v1= df.drop(['Loan_ID','Gender'], axis=1)
df_v1= df_v1.dropna()
print(df_v1)

    Married Dependents     Education Self_Employed  ApplicantIncome  \
1       Yes          1      Graduate            No             4583   
2       Yes          0      Graduate           Yes             3000   
3       Yes          0  Not Graduate            No             2583   
4        No          0      Graduate            No             6000   
5       Yes          2      Graduate           Yes             5417   
..      ...        ...           ...           ...              ...   
609      No          0      Graduate            No             2900   
610     Yes         3+      Graduate            No             4106   
611     Yes          1      Graduate            No             8072   
612     Yes          2      Graduate            No             7583   
613      No          0      Graduate           Yes             4583   

     CoapplicantIncome  LoanAmount  Loan_Amount_Term  Credit_History  \
1               1508.0       128.0             360.0             1.0   
2  

In [4]:
num_variations_dep= df_v1['Dependents'].unique()
print(num_variations_dep)
num_variations_edu= df_v1['Education'].unique()
print(num_variations_edu)
num_variations_self= df_v1['Self_Employed'].unique()
print(num_variations_self)
num_variations_prop= df_v1['Property_Area'].unique()
print(num_variations_prop)
num_variations_cred= df_v1['Credit_History'].unique()
print(num_variations_cred)

['1' '0' '2' '3+']
['Graduate' 'Not Graduate']
['No' 'Yes']
['Rural' 'Urban' 'Semiurban']
[1. 0.]


In [5]:
#removing the text values
def map_to_binary(value):
  return 1 if value == 'Yes' else 0

In [6]:
df_v1['Married_1']= df_v1['Married'].apply(map_to_binary)

print(df_v1)

    Married Dependents     Education Self_Employed  ApplicantIncome  \
1       Yes          1      Graduate            No             4583   
2       Yes          0      Graduate           Yes             3000   
3       Yes          0  Not Graduate            No             2583   
4        No          0      Graduate            No             6000   
5       Yes          2      Graduate           Yes             5417   
..      ...        ...           ...           ...              ...   
609      No          0      Graduate            No             2900   
610     Yes         3+      Graduate            No             4106   
611     Yes          1      Graduate            No             8072   
612     Yes          2      Graduate            No             7583   
613      No          0      Graduate           Yes             4583   

     CoapplicantIncome  LoanAmount  Loan_Amount_Term  Credit_History  \
1               1508.0       128.0             360.0             1.0   
2  

In [7]:
def map_to_binary_edu(value):
  return 1 if value == 'Graduate' else 0

In [8]:
df_v1['Education_1']=df_v1['Education'].apply(map_to_binary_edu)
print(df_v1)

    Married Dependents     Education Self_Employed  ApplicantIncome  \
1       Yes          1      Graduate            No             4583   
2       Yes          0      Graduate           Yes             3000   
3       Yes          0  Not Graduate            No             2583   
4        No          0      Graduate            No             6000   
5       Yes          2      Graduate           Yes             5417   
..      ...        ...           ...           ...              ...   
609      No          0      Graduate            No             2900   
610     Yes         3+      Graduate            No             4106   
611     Yes          1      Graduate            No             8072   
612     Yes          2      Graduate            No             7583   
613      No          0      Graduate           Yes             4583   

     CoapplicantIncome  LoanAmount  Loan_Amount_Term  Credit_History  \
1               1508.0       128.0             360.0             1.0   
2  

In [9]:
def map_to_binary_self(value):
  return 1 if value == 'No' else 0

In [10]:
df_v1['Self_Employed_1']=df_v1['Self_Employed'].apply(map_to_binary_self)
df_v1

,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,Married_1,Education_1,Self_Employed_1
1,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N,1,1,1
2,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y,1,1,0
3,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y,1,0,1
4,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y,0,1,1
5,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y,0,1,1
610,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y,1,1,1
611,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y,1,1,1
612,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y,1,1,1


In [11]:
def map_to_binary_property(value):
  if value == 'Urban':
    return 1
  elif value == 'Semiurban':
    return 2
  else:
    return 0

In [12]:
df_v1['Property_Area_1']= df_v1['Property_Area'].apply(map_to_binary_property)
df_v1

,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,Married_1,Education_1,Self_Employed_1,Property_Area_1
1,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N,1,1,1,0
2,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y,1,1,0,1
3,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y,1,0,1,1
4,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y,0,1,1,1
5,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y,0,1,1,0
610,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y,1,1,1,0
611,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y,1,1,1,1
612,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y,1,1,1,1


In [13]:
df_v1['Dependents_1']=df_v1['Dependents'].replace('3+', 3)
df_v1

,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,Married_1,Education_1,Self_Employed_1,Property_Area_1,Dependents_1
1,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N,1,1,1,0,1
2,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y,1,1,0,1,0
3,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y,1,0,1,1,0
4,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y,0,1,1,1,0
5,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y,1,1,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y,0,1,1,0,0
610,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y,1,1,1,0,3
611,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y,1,1,1,1,1
612,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y,1,1,1,1,2


In [14]:
def map_to_binary_loan(value):
  return 1 if value == 'Y' else 0

In [15]:
df_v1['Loan_Status_1']=df_v1['Loan_Status'].apply(map_to_binary_loan)
df_v1

,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,Married_1,Education_1,Self_Employed_1,Property_Area_1,Dependents_1,Loan_Status_1
1,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N,1,1,1,0,1,0
2,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y,1,1,0,1,0,1
3,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y,1,0,1,1,0,1
4,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y,0,1,1,1,0,1
5,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y,1,1,0,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y,0,1,1,0,0,1
610,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y,1,1,1,0,3,1
611,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y,1,1,1,1,1,1
612,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y,1,1,1,1,2,1


In [89]:
df_v2= df_v1.drop(['Married','Education','Self_Employed','Property_Area','Loan_Status','Dependents'], axis=1)
df_v2['Dependents_1']=df_v2['Dependents_1'].astype(int)
df_v2

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Married_1,Education_1,Self_Employed_1,Property_Area_1,Dependents_1,Loan_Status_1
1,4583,1508.0,128.0,360.0,1.0,1,1,1,0,1,0
2,3000,0.0,66.0,360.0,1.0,1,1,0,1,0,1
3,2583,2358.0,120.0,360.0,1.0,1,0,1,1,0,1
4,6000,0.0,141.0,360.0,1.0,0,1,1,1,0,1
5,5417,4196.0,267.0,360.0,1.0,1,1,0,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...
609,2900,0.0,71.0,360.0,1.0,0,1,1,0,0,1
610,4106,0.0,40.0,180.0,1.0,1,1,1,0,3,1
611,8072,240.0,253.0,360.0,1.0,1,1,1,1,1,1
612,7583,0.0,187.0,360.0,1.0,1,1,1,1,2,1


In [16]:
# #import the dataset
# data = pd.read_csv('train.csv')

In [17]:
# #2. Checking the data set
# data.head()

In [18]:
# data.tail()

In [19]:
# #3. Find the number of rows and columns in the dataset
# data.shape
# print("Number of Rows",data.shape[0])
# print("Number of Columns",data.shape[1])

In [20]:
# #4. Breakdown of the dataset
# data.info()

In [21]:
# #5. Check for null values in the dataset
# data.isnull().sum()

In [22]:
# data.isnull().sum()*100 / len(data)

In [23]:
# #6. Handling the missing values in the dataset
# data = data.drop('Loan_ID',axis=1)

In [24]:
# data.head(1)

In [25]:
# columns = ['Gender','Dependents','LoanAmount','Loan_Amount_Term']
# data = data.dropna(subset=columns)

In [26]:
# data.isnull().sum()*100 / len(data)

In [27]:
# data['Self_Employed'].mode()[0]

In [28]:
# data['Self_Employed'] =data['Self_Employed'].fillna(data['Self_Employed'].mode()[0])

In [29]:
# data.isnull().sum()*100 / len(data)

In [30]:
# data['Gender'].unique()

In [31]:
# data['Self_Employed'].unique()

In [32]:
# data['Credit_History'].mode()[0]

In [33]:
# data['Credit_History'] =data['Credit_History'].fillna(data['Credit_History'].mode()[0])

In [34]:
# data.isnull().sum()*100 / len(data)

In [35]:
# 7. Handling Categorical Columns

In [36]:
# data.sample(5)

In [37]:
# data['Dependents'] =data['Dependents'].replace(to_replace="3+",value='4')

In [38]:
# data['Dependents'].unique()

In [39]:
# data['Loan_Status'].unique()

In [40]:
# data['Gender'] = data['Gender'].map({'Male':1,'Female':0}).astype('int')
# data['Married'] = data['Married'].map({'Yes':1,'No':0}).astype('int')
# data['Education'] = data['Education'].map({'Graduate':1,'Not Graduate':0}).astype('int')
# data['Self_Employed'] = data['Self_Employed'].map({'Yes':1,'No':0}).astype('int')
# data['Property_Area'] = data['Property_Area'].map({'Rural':0,'Semiurban':2,'Urban':1}).astype('int')
# data['Loan_Status'] = data['Loan_Status'].map({'Y':1,'N':0}).astype('int')

In [41]:
# data.head()

In [42]:
#8. Separating the features and target variable into x & y

In [43]:
# X = data.drop('Loan_Status',axis=1)

In [44]:
# y = data['Loan_Status']

In [45]:
# y

In [46]:
# 9. Feature Scaling

In [47]:
# data.head()

In [48]:
# cols = ['ApplicantIncome','CoapplicantIncome','LoanAmount','Loan_Amount_Term']

In [49]:
# from sklearn.preprocessing import StandardScaler
# st = StandardScaler()
# X[cols]=st.fit_transform(X[cols])

In [50]:
# X

In [51]:
#10. Performance of Different Machine Learning Models + Dividing the dataset into Test and Train sets + applying K-fold Cross Validation

In [52]:
# #libraries
# from sklearn.model_selection import train_test_split
# from sklearn.model_selection import cross_val_score
# from sklearn.metrics import accuracy_score
# import numpy as np

In [53]:
# model_df={}
# def model_val(model,X,y):
#     X_train,X_test,y_train,y_test=train_test_split(X,y,
#                                                    test_size=0.20,
#                                                    random_state=42)
#     model.fit(X_train,y_train)
#     y_pred=model.predict(X_test)
#     print(f"{model} accuracy is {accuracy_score(y_test,y_pred)}")

#     score = cross_val_score(model,X,y,cv=5)
#     print(f"{model} Avg cross val score is {np.mean(score)}")
#     model_df[model]=round(np.mean(score)*100,2)

In [54]:
# #Logistic Regression model
# from sklearn.linear_model import LogisticRegression
# model = LogisticRegression()
# model_val(model,X,y)

In [55]:
# #SVC model
# from sklearn import svm
# model = svm.SVC()
# model_val(model,X,y)

In [56]:
# #Decision Tree Classifier model
# from sklearn.tree import DecisionTreeClassifier
# model = DecisionTreeClassifier()
# model_val(model,X,y)

In [57]:
# #Random Forest Classifier model
# from sklearn.ensemble import RandomForestClassifier
# model =RandomForestClassifier()
# model_val(model,X,y)

In [58]:
# #Gradient Boosting Classifier model
# from sklearn.ensemble import GradientBoostingClassifier
# model =GradientBoostingClassifier()
# model_val(model,X,y)

In [59]:
# #Average Cross Validation of each model as a percentage is evaluated and stored in "model_df" dictionary
# model_df

In [60]:
# 11. Hyperparameter Tuning

In [61]:
# from sklearn.model_selection import RandomizedSearchCV

In [62]:
# #Logistic Regression Tuning
# log_reg_grid={"C":np.logspace(-4,4,20),
#              "solver":['liblinear']}

In [63]:
# rs_log_reg=RandomizedSearchCV(LogisticRegression(),
#                    param_distributions=log_reg_grid,
#                   n_iter=20,cv=5,verbose=True)

In [64]:
# rs_log_reg.fit(X,y)

In [65]:
# rs_log_reg.best_score_

In [66]:
# rs_log_reg.best_params_

In [67]:
# #SVC Tuning
# svc_grid = {'C':[0.25,0.50,0.75,1],"kernel":["linear"]}

In [68]:
# rs_svc=RandomizedSearchCV(svm.SVC(),
#                   param_distributions=svc_grid,
#                    cv=5,
#                    n_iter=20,
#                   verbose=True)

In [69]:
# rs_svc.fit(X,y)

In [70]:
# rs_svc.best_score_

In [71]:
# rs_svc.best_params_

In [72]:
# #Random Forest Classifier Tuning
# RandomForestClassifier()

# rf_grid={'n_estimators':np.arange(10,1000,10),
#   'max_features':['auto','sqrt'],
#  'max_depth':[None,3,5,10,20,30],
#  'min_samples_split':[2,5,20,50,100],
#  'min_samples_leaf':[1,2,5,10]
#  }

In [73]:
# rs_rf=RandomizedSearchCV(RandomForestClassifier(),
#                   param_distributions=rf_grid,
#                    cv=5,
#                    n_iter=20,
#                   verbose=True)

In [74]:
# rs_rf.fit(X,y)

In [75]:
# rs_rf.best_score_

In [76]:
# rs_rf.best_params_

#Saving The Model

In [77]:
# X = data.drop('Loan_Status',axis=1)
# y = data['Loan_Status']

In [78]:
# rf = RandomForestClassifier(n_estimators=270,
#  min_samples_split=5,
#  min_samples_leaf=5,
#  max_features='sqrt',
#  max_depth=5)

In [79]:
# rf.fit(X,y)

In [80]:
# import joblib

In [81]:
# joblib.dump(rf,'loan_status_predict')

In [82]:
# model = joblib.load('loan_status_predict')

In [83]:
# data = {
#     'Gender': [1],
#     'Married': [0],
#     'Dependents': [2],
#     'Education': [1],
#     'Self_Employed': [0],
#     'ApplicantIncome': [4923],
#     'CoapplicantIncome': [0.0],
#     'LoanAmount': [166],
#     'Loan_Amount_Term': [360],
#     'Credit_History': [0],
#     'Property_Area': [1]
# }

In [84]:
# df = pd.DataFrame(data)

In [85]:
# df

In [86]:
# model.predict(df)

In [87]:
# result = model.predict(df)

In [88]:
# if result==1:
#     print("Loan Approved")
# else:
#     print("Loan Not Approved")